# Exercise 5-3: Get data from a database

In [1]:
# Single cell to do all importing

# Import pandas and create an alias
import pandas as pd

# Import request from the urllib module
from urllib import request

# Import ZipFile from the zipfile module
from zipfile import ZipFile

# Import sqlite3
import sqlite3


In [2]:
fires_url = 'https://www.fs.usda.gov/rds/archive/products/RDS-2013-0009.4/RDS-2013-0009.4_SQLITE.zip'

In [3]:
# Step 2, See figure 5-3, Download the zip file to the local drive
request.urlretrieve(fires_url, filename = 'RDS_sqlite.zip')

('RDS_sqlite.zip', <http.client.HTTPMessage at 0x7f620a7c54b0>)

In [4]:
# Step 3 - See figure 5-4, Extract the files from the zip file and diplsy the file names

file_names = list()
with ZipFile('RDS_sqlite.zip', mode = 'r') as zip:
    zip.extractall()
    for file in zip.infolist():
        file_names.append(file.filename)
        print(file.filename, file.compress_size, file.file_size)
        

Data/FPA_FOD_20170508.sqlite 173776108 795785216
Data/ 0 0
_metadata_RDS-2013-0009.4.xml 11816 51410
Supplements/FPA_FOD_Source_List.pdf 74137 109336
Supplements/ 0 0
_fileindex_RDS-2013-0009.4.html 1098 4398
_metadata_RDS-2013-0009.4.html 13803 88383


## Run queries against the database

In [5]:
# Step 5 - See figure 5-5, create a connection object and a cursor object for the DB file.
#          Then run a query to list the table names.

fires_con = sqlite3.connect('Data/FPA_FOD_20170508.sqlite')
fires_cur = fires_con.cursor()

# Run a query to list the table names.
fires_cur.execute(
          'SELECT name FROM sqlite_master WHERE type = "table"').fetchall()

[('spatial_ref_sys',),
 ('spatialite_history',),
 ('sqlite_sequence',),
 ('geometry_columns',),
 ('spatial_ref_sys_aux',),
 ('views_geometry_columns',),
 ('virts_geometry_columns',),
 ('geometry_columns_statistics',),
 ('views_geometry_columns_statistics',),
 ('virts_geometry_columns_statistics',),
 ('geometry_columns_field_infos',),
 ('views_geometry_columns_field_infos',),
 ('virts_geometry_columns_field_infos',),
 ('geometry_columns_time',),
 ('geometry_columns_auth',),
 ('views_geometry_columns_auth',),
 ('virts_geometry_columns_auth',),
 ('sql_statements_log',),
 ('SpatialIndex',),
 ('ElementaryGeometries',),
 ('KNN',),
 ('Fires',),
 ('idx_Fires_Shape',),
 ('idx_Fires_Shape_node',),
 ('idx_Fires_Shape_rowid',),
 ('idx_Fires_Shape_parent',),
 ('NWCG_UnitIDActive_20170109',)]

In [6]:
# Step 6 - See figure 5-6, run a query that lists the information about the columns in the Fires table.

fires_cur.execute('PRAGMA table_info(Fires)').fetchall()

[(0, 'OBJECTID', 'INTEGER', 1, None, 1),
 (1, 'FOD_ID', 'int32', 0, None, 0),
 (2, 'FPA_ID', 'text(100)', 0, None, 0),
 (3, 'SOURCE_SYSTEM_TYPE', 'text(255)', 0, None, 0),
 (4, 'SOURCE_SYSTEM', 'text(30)', 0, None, 0),
 (5, 'NWCG_REPORTING_AGENCY', 'text(255)', 0, None, 0),
 (6, 'NWCG_REPORTING_UNIT_ID', 'text(255)', 0, None, 0),
 (7, 'NWCG_REPORTING_UNIT_NAME', 'text(255)', 0, None, 0),
 (8, 'SOURCE_REPORTING_UNIT', 'text(30)', 0, None, 0),
 (9, 'SOURCE_REPORTING_UNIT_NAME', 'text(255)', 0, None, 0),
 (10, 'LOCAL_FIRE_REPORT_ID', 'text(255)', 0, None, 0),
 (11, 'LOCAL_INCIDENT_ID', 'text(255)', 0, None, 0),
 (12, 'FIRE_CODE', 'text(10)', 0, None, 0),
 (13, 'FIRE_NAME', 'text(255)', 0, None, 0),
 (14, 'ICS_209_INCIDENT_NUMBER', 'text(255)', 0, None, 0),
 (15, 'ICS_209_NAME', 'text(255)', 0, None, 0),
 (16, 'MTBS_ID', 'text(255)', 0, None, 0),
 (17, 'MTBS_FIRE_NAME', 'text(50)', 0, None, 0),
 (18, 'COMPLEX_NAME', 'text(255)', 0, None, 0),
 (19, 'FIRE_YEAR', 'int16', 0, None, 0),
 (20, '

## Read the results of a SQL query into a DataFrame

In [7]:
# Step 7, See figure 5-6, use read_sql_query() method to read data into a DataFrame.
# 

fires = pd.read_sql_query(
        ''' SELECT STATE, FIRE_YEAR, DATETIME(DISCOVERY_DATE) AS DISCOVERY_DATE,
        FIRE_NAME, FIRE_SIZE, LATITUDE, LONGITUDE
        FROM Fires''', fires_con)


In [8]:
# Step 8, Display the first 5 rows of the fires DataFrame.

fires.head()

,STATE,FIRE_YEAR,DISCOVERY_DATE,FIRE_NAME,FIRE_SIZE,LATITUDE,LONGITUDE
0,CA,2005,2005-02-02 00:00:00,FOUNTAIN,0.10,40.036944,-121.005833
1,CA,2004,2004-05-12 00:00:00,PIGEON,0.25,38.933056,-120.404444
2,CA,2004,2004-05-31 00:00:00,SLACK,0.10,38.984167,-120.735556
3,CA,2004,2004-06-28 00:00:00,DEER,0.10,38.559167,-119.913333
4,CA,2004,2004-06-28 00:00:00,STEVENOT,0.10,38.559167,-119.933056


In [9]:
# Step 9, Display the info() information for the fires DataFrame.

fires.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1880465 entries, 0 to 1880464
Data columns (total 7 columns):
 #   Column          Dtype  
---  ------          -----  
 0   STATE           object 
 1   FIRE_YEAR       int64  
 2   DISCOVERY_DATE  object 
 3   FIRE_NAME       object 
 4   FIRE_SIZE       float64
 5   LATITUDE        float64
 6   LONGITUDE       float64
dtypes: float64(3), int64(1), object(3)
memory usage: 100.4+ MB


In [10]:
# Create an Arizon only DataFrame and then display the first 5 rows of the az_fires DataFrame,
#    sorted by FIRE_SIZE, in decending order.

az_fires = fires.query('STATE == "AZ"')

az_fires.sort_values('FIRE_SIZE', ascending = False).head()

,STATE,FIRE_YEAR,DISCOVERY_DATE,FIRE_NAME,FIRE_SIZE,LATITUDE,LONGITUDE
1459664,AZ,2011,2011-05-29 00:00:00,WALLOW,538049.0,33.606111,-109.449722
297291,AZ,2002,2002-06-18 00:00:00,RODEO,259158.9,34.108400,-110.485900
1059558,AZ,2005,2005-06-21 00:00:00,HUMBOLDT,248310.0,33.978333,-111.822778
1460217,AZ,2011,2011-05-08 00:00:00,HORSESHOE 2,222954.0,31.819722,-109.211111
297292,AZ,2002,2002-06-20 00:00:00,CHEDISKI,209704.1,34.136200,-110.702900


In [11]:
az_fires.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71586 entries, 75 to 1860016
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   STATE           71586 non-null  object 
 1   FIRE_YEAR       71586 non-null  int64  
 2   DISCOVERY_DATE  71586 non-null  object 
 3   FIRE_NAME       60986 non-null  object 
 4   FIRE_SIZE       71586 non-null  float64
 5   LATITUDE        71586 non-null  float64
 6   LONGITUDE       71586 non-null  float64
dtypes: float64(3), int64(1), object(3)
memory usage: 4.4+ MB
